# Trainingspipeline 30

- balanciertes Korpus
- ohne Bigramme
- vector_size: 300
- window: 10
- seed: 1

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
df = pd.read_csv('../Korpus/Korpus/corpus_balanced.csv', sep=',', encoding='utf-8')

In [4]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text,cleaned_tokenized_text
0,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0,iv . 4 . rima del arcadi : aglauro cidonia ( f...,"[['iv'], [], ['rima', 'arcadi', 'aglauro', 'ci..."
1,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,"﻿iv . 4 . Ludovico Antonio muratori , antichit...","[['iv'], [], ['ludovico', 'antonio', 'muratori..."
2,Personali.IV.5.Testo.txt,MIDIA,Lorenzo Magalotti,Lettere odorose (1693-1705),1700.0,1700-1750,personale,"IV. 5. Lorenzo Magalotti, Lettere odorose (169...",8374.0,"iv . 5 . Lorenzo magalotti , lettere odoroso (...","[['iv'], [], ['lorenzo', 'magalotti', 'lettere..."
3,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0,[ proemio ] \n prendere a scrivere il mio vita...,"[['proemio', 'prendere', 'scrivere', 'vita', '..."
4,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0,"iv . 4 . Vincenzo da filicaia , lettere inedit...","[['iv'], [], ['vincenzo', 'filicaia', 'lettere..."


In [5]:
df.shape

(743763, 11)

In [6]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [7]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [8]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1975']
df10 = df_periods['1976-2000']
df11 = df_periods['2001-2010']
df12 = df_periods['2011-2016']
df13 = df_periods['2017-2021']

In [9]:
df9.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text,cleaned_tokenized_text
126313,LISRodari3.txt,LIS,Gianno Rodari,La questione dei fumetti,1951.0,1951-1975,stampa,"Caro Direttore , ho letto nell ' ultimo numero...",1510.0,"caro direttore , avere leggere nell ' ultimo n...","[['caro', 'direttore', 'avere', 'leggere', 'ul..."
126314,LISJotti1.txt,LIS,Nilde Jotti,La questione dei fumetti,1951.0,1951-1975,stampa,Il dibattito sulla stampa a fumetti per i raga...,2785.0,il dibattito sulla stampa a fumetto per il rag...,"[['dibattito', 'stampa', 'fumetto', 'ragazzo',..."
126315,LLAlbertelli1.txt,Liber Liber,Pilo Albertelli,Rousseau,1951.0,1951-1975,prosa letteraria,﻿Pilo Albertelli\nRousseau\n\n Nacque il 28 g...,4894.0,﻿pilo albertelli \n rousseau \n\n nascere il...,"[['pilo', 'albertelli', 'rousseau', 'nascere',..."
126316,LISManacorda1.txt,LIS,Gastone Manacorda,Il Partito e la sua funzione di guida nel camp...,1951.0,1951-1975,stampa,"Al partito , nel suo rigoglioso sviluppo , seg...",3460.0,"al partito , nel suo rigoglioso sviluppo , seg...","[['partito', 'rigoglioso', 'sviluppo', 'seguit..."
126317,LISBianchi1.txt,LIS,Ranuccio Bianchi Bandinelli,Il nostro lavoro nella scuola,1951.0,1951-1975,stampa,"Come in tutti i congressi , anche nel VII Cong...",2898.0,"come in tutto il congresso , anche nel vii con...","[['congresso', 'vii', 'congresso', 'p'], ['tes..."


## Training von Word2Vec

In [10]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [11]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 10                        # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1

In [12]:
# Ordner anlegen zum Abspeichern von trainierten Modellen

if not os.path.exists('../trained_models'):
    os.makedirs('../trained_models')

### Zeitraum 1: 1700-1750

In [13]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [14]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 21.8 s


In [15]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 4min 18s


In [16]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('lodigiano', 0.5615582466125488),
 ('costernazione', 0.5529147982597351),
 ('spavento', 0.5432156324386597),
 ('turchesche', 0.5418321490287781),
 ('riempiè', 0.5404583811759949),
 ('implacabilità', 0.5221208333969116),
 ('avanzò', 0.5218067765235901),
 ('alviano', 0.5214826464653015),
 ('impadronitosi', 0.5132208466529846),
 ('barbari', 0.5108266472816467)]

In [17]:
# trainiertes Modell speichern

w2v1.save(os.path.join('../trained_models/Word2Vec30', '30w2v1.model'))

### Zeitraum 2: 1751-1800

In [18]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [19]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 25.4 s


In [20]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 57s


In [21]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('formidabil', 0.614395797252655),
 ('ostile', 0.6111111044883728),
 ('crociati', 0.5993159413337708),
 ('timur', 0.5929927229881287),
 ('inspirare', 0.5877782106399536),
 ('legione', 0.5870984792709351),
 ('devastazione', 0.5841709971427917),
 ('avvicinarsi', 0.5518136620521545),
 ('avversione', 0.5478644371032715),
 ('tribù', 0.5423879027366638)]

In [22]:
w2v2.save(os.path.join('../trained_models/Word2Vec30', '30w2v2.model'))

### Zeitraum 3: 1801-1825

In [23]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [24]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 22.5 s


In [25]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 4min 1s


In [26]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('repentino', 0.700520396232605),
 ('sovrannaturale', 0.6435088515281677),
 ('aspettò', 0.6315177083015442),
 ('fremè', 0.6243984699249268),
 ('impadronisce', 0.6171139478683472),
 ('sorpresa', 0.6126025319099426),
 ('innalza', 0.6068622469902039),
 ('imaginari', 0.5984677672386169),
 ('rammarico', 0.5919321775436401),
 ('abbattimento', 0.5886495113372803)]

In [27]:
w2v3.save(os.path.join('../trained_models/Word2Vec30', '30w2v3.model'))

### Zeitraum 4: 1826-1850

In [28]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [29]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 21.8 s


In [30]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 58s


In [31]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('panico', 0.543762743473053),
 ('bentosto', 0.5170122981071472),
 ('inspirare', 0.5107207894325256),
 ('villetard', 0.482183039188385),
 ('riurto', 0.47408032417297363),
 ('scompiglio', 0.4709429442882538),
 ('dissipare', 0.46923819184303284),
 ('accostava', 0.4678434729576111),
 ('incusso', 0.4651920199394226),
 ('aristocrazìa', 0.4642694294452667)]

In [32]:
w2v4.save(os.path.join('../trained_models/Word2Vec30', '30w2v4.model'))

### Zeitraum 5: 1851-1875

In [33]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [34]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 23.9 s


In [35]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 38s


In [36]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('sgomento', 0.5722348690032959),
 ('spavento', 0.54362952709198),
 ('orrore', 0.5253002643585205),
 ('rabbia', 0.5158381462097168),
 ('gioja', 0.5135176181793213),
 ('rammarichío', 0.5075597763061523),
 ('raccapriccio', 0.5061434507369995),
 ('tradito', 0.5018460750579834),
 ('agonia', 0.4989558458328247),
 ('atterriti', 0.4980342984199524)]

In [37]:
w2v5.save(os.path.join('../trained_models/Word2Vec30', '30w2v5.model'))

### Zeitraum 6: 1876-1900

In [38]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [39]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 26.1 s


In [40]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 23s


In [41]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.49233606457710266),
 ('sgomento', 0.4664325714111328),
 ('calamità', 0.45987528562545776),
 ('orrore', 0.4558510482311249),
 ('sbigottimento', 0.44617393612861633),
 ('raccapriccio', 0.43962791562080383),
 ('superstizioso', 0.4382532238960266),
 ('forsennato', 0.4303641617298126),
 ('carneficina', 0.43014198541641235),
 ('assalire', 0.42371997237205505)]

In [42]:
w2v6.save(os.path.join('../trained_models/Word2Vec30', '30w2v6.model'))

### Zeitraum 7: 1901-1925

In [43]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [44]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 27.5 s


In [45]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 25s


In [46]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('gelare', 0.4697500169277191),
 ('smarrimento', 0.4620377719402313),
 ('ribrezzo', 0.46144673228263855),
 ('orrore', 0.46043723821640015),
 ('selvaggiamente', 0.4515904486179352),
 ('sbalordimento', 0.4502129554748535),
 ('giovanezza', 0.4491518437862396),
 ('inopinatamente', 0.4482017755508423),
 ('sfuggito', 0.4477081596851349),
 ('accesso', 0.44765958189964294)]

In [47]:
w2v7.save(os.path.join('../trained_models/Word2Vec30', '30w2v7.model'))

### Zeitraum 8: 1926-1950

In [48]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [49]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 24.6 s


In [50]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 42s


In [51]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('rabbico', 0.562032163143158),
 ('virus', 0.5619686841964722),
 ('paralizzare', 0.4947842061519623),
 ('frammettendosi', 0.4936181306838989),
 ('innestato', 0.49003320932388306),
 ('impietrire', 0.48735207319259644),
 ('semispento', 0.48045191168785095),
 ('soffocandomi', 0.47972235083580017),
 ('pettirosso', 0.47964251041412354),
 ('conversa', 0.4790685772895813)]

In [52]:
w2v8.save(os.path.join('../trained_models/Word2Vec30', '30w2v8.model'))

### Zeitraum 9: 1951-1975

In [53]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [54]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 25.3 s


In [55]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 52s


In [56]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('malefico', 0.6287841796875),
 ('insano', 0.6057776212692261),
 ('spasmodicamente', 0.604286253452301),
 ('orrore', 0.6025545001029968),
 ('assopita', 0.5936296582221985),
 ('raccapriccio', 0.5830296277999878),
 ('smarrimento', 0.5782256722450256),
 ('inorridito', 0.5777332186698914),
 ('lussuria', 0.5761638283729553),
 ('opprime', 0.5701215267181396)]

In [57]:
w2v9.save(os.path.join('../trained_models/Word2Vec30', '30w2v9.model'))

### Zeitraum 10: 1976-2000

In [58]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [59]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 28.4 s


In [60]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 3min 37s


In [61]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('staliniano', 0.5717442631721497),
 ('terrorista', 0.5644408464431763),
 ('fondamentalista', 0.5635892152786255),
 ('sanguinario', 0.5524724721908569),
 ('terrorismo', 0.5483409762382507),
 ('incubo', 0.5468292832374573),
 ('autobomba', 0.5467087030410767),
 ('pol', 0.5458354353904724),
 ('deportazione', 0.5430185794830322),
 ('fanatico', 0.539924144744873)]

In [62]:
w2v10.save(os.path.join('../trained_models/Word2Vec30', '30w2v10.model'))

### Zeitraum 11: 2001-2010

In [63]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [64]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 22.1 s


In [65]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 4min 30s


In [66]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('orrore', 0.5889381170272827),
 ('oppressione', 0.5729219913482666),
 ('malvagità', 0.5616228580474854),
 ('atroce', 0.5597056746482849),
 ('barbarie', 0.5489183068275452),
 ('fanatismo', 0.5480705499649048),
 ('carestia', 0.5477238893508911),
 ('didi', 0.5462226867675781),
 ('carneficina', 0.5446843504905701),
 ('ceceno', 0.5436684489250183)]

In [67]:
w2v11.save(os.path.join('../trained_models/Word2Vec30', '30w2v11.model'))

### Zeitraum 12: 2011-2016

In [68]:
text12 = ''

for i in df12.lemmatized_text:
    text12+= i

In [69]:
%%time
sentences12 = tokenize_text(text12)

Wall time: 20.2 s


In [70]:
%%time
w2v12 = Word2Vec(sentences=sentences12,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 4min 43s


In [71]:
w2v12.wv.most_similar(positive=['terrore'], topn=10)

[('ripiombare', 0.6142153143882751),
 ('incutere', 0.6073843240737915),
 ('odio', 0.5898599624633789),
 ('desolazione', 0.5756441354751587),
 ('cappa', 0.5743535757064819),
 ('seminare', 0.5727759003639221),
 ('sopraffazione', 0.5697144269943237),
 ('ferocia', 0.5689697861671448),
 ('sottomissione', 0.5672918558120728),
 ('plum', 0.5656874179840088)]

In [72]:
w2v12.save(os.path.join('../trained_models/Word2Vec30', '30w2v12.model'))

### Zeitraum 13: 2017-2021

In [73]:
text13 = ''

for i in df13.lemmatized_text:
    text13+= i

In [74]:
%%time
sentences13 = tokenize_text(text13)

Wall time: 18.9 s


In [75]:
%%time
w2v13 = Word2Vec(sentences=sentences13,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 5min


In [76]:
w2v13.wv.most_similar(positive=['terrore'], topn=10)

[('incutere', 0.6539342403411865),
 ('orrore', 0.6452838182449341),
 ('seminare', 0.628483235836029),
 ('impotenza', 0.6272907853126526),
 ('disorientamento', 0.6267272233963013),
 ('angoscia', 0.6199542880058289),
 ('paura', 0.6124224066734314),
 ('oppressione', 0.6107749342918396),
 ('frenesia', 0.6096921563148499),
 ('crudeltà', 0.6054523587226868)]

In [77]:
w2v13.save(os.path.join('../trained_models/Word2Vec30', '30w2v13.model'))